# Trabalho de Engenharia de Dados II

## Membros
* Clairton Carneiro Luz - T6
* Lairton da Silva Mendes - T6
* Paulo Roberto Mesquita da Silva - T5

## 1 Configurando ambiente

### 1.1 Instalando dependências

In [2]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2 Coleta dos dados

### 2.1 Baixando dataset

In [3]:

!mkdir -p datasets
# if not exist, download the dataset
import os
import gdown

if not os.path.exists('datasets/archive.zip'):
    url = 'https://drive.google.com/uc?id=17lmw3IkAM4T71chEJkxYNQSB-Ky9s14-'
    output = 'datasets/archive.zip'
    gdown.download(url, output, quiet=False)

### 2.2 Extraindo dataset

In [4]:
#  if not exist, unzip the dataset
import zipfile
import os

if not os.path.exists('datasets/past-data.csv'):
    with zipfile.ZipFile('datasets/archive.zip', 'r') as zip_ref:
        zip_ref.extractall('datasets/')

### 2.3 Carregando dataset

In [5]:
# Imprime leganda do dataset
with open('datasets/headers-details.txt', 'r') as file:
    print(file.read())


Key to data:

Season = League Season
Div = League Division
Date = Match Date (dd/mm/yy)
Time = Time of match kick off
HomeTeam = Home Team
AwayTeam = Away Team
FTHG = Full Time Home Team Goals
FTAG = Full Time Away Team Goals
FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
HTHG = Half Time Home Team Goals
HTAG = Half Time Away Team Goals
HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

Match Statistics (where available)
Referee = Match Referee
HS = Home Team Shots
AS = Away Team Shots
HST = Home Team Shots on Target
AST = Away Team Shots on Target
HC = Home Team Corners
AC = Away Team Corners
HF = Home Team Fouls Committed
AF = Away Team Fouls Committed
HY = Home Team Yellow Cards
AY = Away Team Yellow Cards
HR = Home Team Red Cards
AR = Away Team Red Cards


In [6]:
# Carregando = 'datasets/onlinefoods.csv' usando spark que está rodando localmente using componse.yml

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder \
    .appName("Football Data | Top 5 European Leagues") \
    .master("local[2]") \
    .getOrCreate()

sc = spark.sparkContext

df=spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv('./datasets/past-data.csv')

df.show(5)


24/04/05 20:06:36 WARN Utils: Your hostname, MacBook-Air-de-Clairton.local resolves to a loopback address: 127.0.0.1; using 10.50.30.151 instead (on interface en0)
24/04/05 20:06:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/05 20:06:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------+--------------+----------+-------------------+--------------+-------------+----+----+---+----+----+---+--------+----+----+----+---+----+----+----+---+---+---+---+---+
|Season|           Div|      Date|               Time|      HomeTeam|     AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR| Referee|  HS|  AS| HST|AST|  HF|  AF|  HC| AC| HY| AY| HR| AR|
+------+--------------+----------+-------------------+--------------+-------------+----+----+---+----+----+---+--------+----+----+----+---+----+----+----+---+---+---+---+---+
| 22-23|Premier League|05/08/2022|2024-04-05 20:00:00|Crystal Palace|      Arsenal| 0.0| 2.0|  A| 0.0| 1.0|  A|A Taylor|10.0|10.0| 2.0|2.0|16.0|11.0| 3.0|5.0|1.0|2.0|0.0|0.0|
| 22-23|Premier League|06/08/2022|2024-04-05 12:30:00|        Fulham|    Liverpool| 2.0| 2.0|  D| 1.0| 0.0|  H|A Madley| 9.0|11.0| 3.0|4.0| 7.0| 9.0| 4.0|4.0|2.0|0.0|0.0|0.0|
| 22-23|Premier League|06/08/2022|2024-04-05 15:00:00|   Bournemouth|  Aston Villa| 2.0| 0.0|  H| 1.0| 0.0|  H|P Bankes| 7.0|

## 3 Limpeza e pré-processamento dos dados

### 3.1 Traduzindo colunas

In [7]:
# traduzindo colunas para pt-br
df = df.withColumnRenamed('Season', 'temporada')
df = df.withColumnRenamed('Div', 'divisao')
df = df.withColumnRenamed('Date', 'data')
df = df.withColumnRenamed('Time', 'inicio_partida')
df = df.withColumnRenamed('HomeTeam', 'time_casa')
df = df.withColumnRenamed('AwayTeam', 'time_fora')
df = df.withColumnRenamed('FTHG', 'gols_time_casa')
df = df.withColumnRenamed('FTAG', 'gols_time_fora')
df = df.withColumnRenamed('FTR', 'resultado')
df = df.withColumnRenamed('HTHG', 'time_casa_gols_intervalo')
df = df.withColumnRenamed('HTAG', 'time_fora_gols_intervalo')
df = df.withColumnRenamed('HTR', 'resultado_intervalo')
df = df.withColumnRenamed('Referee', 'arbitro')
df = df.withColumnRenamed('HS', 'chutes_casa')
df = df.withColumnRenamed('AS', 'chutes_fora')
df = df.withColumnRenamed('HST', 'chutes_no_gol_casa')
df = df.withColumnRenamed('AST', 'chutes_no_gol_fora')
df = df.withColumnRenamed('HC', 'escanteios_casa')
df = df.withColumnRenamed('AC', 'escanteios_fora')
df = df.withColumnRenamed('HF', 'faltas_casa')
df = df.withColumnRenamed('AF', 'faltas_fora')
df = df.withColumnRenamed('HY', 'cartoes_amarelos_casa')
df = df.withColumnRenamed('AY', 'cartoes_amarelos_fora')
df = df.withColumnRenamed('HR', 'cartoes_vermelhos_casa')
df = df.withColumnRenamed('AR', 'cartoes_vermelhos_fora')

df.show(5)


+---------+--------------+----------+-------------------+--------------+-------------+--------------+--------------+---------+------------------------+------------------------+-------------------+--------+-----------+-----------+------------------+------------------+-----------+-----------+---------------+---------------+---------------------+---------------------+----------------------+----------------------+
|temporada|       divisao|      data|     inicio_partida|     time_casa|    time_fora|gols_time_casa|gols_time_fora|resultado|time_casa_gols_intervalo|time_fora_gols_intervalo|resultado_intervalo| arbitro|chutes_casa|chutes_fora|chutes_no_gol_casa|chutes_no_gol_fora|faltas_casa|faltas_fora|escanteios_casa|escanteios_fora|cartoes_amarelos_casa|cartoes_amarelos_fora|cartoes_vermelhos_casa|cartoes_vermelhos_fora|
+---------+--------------+----------+-------------------+--------------+-------------+--------------+--------------+---------+------------------------+---------------------

### 3.2 Traduzindo resultados das partidadas

In [8]:
# traduzindo resultados

from pyspark.sql.functions import when

df = df.withColumn('resultado', when(df['resultado'] == 'A', 'Fora') \
    .when(df['resultado'] == 'D', 'Empate') \
    .when(df['resultado'] == 'H', 'Casa') \
    .otherwise(df['resultado']))

df = df.withColumn('resultado_intervalo', when(df['resultado_intervalo'] == 'A', 'Fora') \
    .when(df['resultado_intervalo'] == 'D', 'Empate') \
    .when(df['resultado_intervalo'] == 'H', 'Casa') \
    .otherwise(df['resultado_intervalo']))

df.show(5)


+---------+--------------+----------+-------------------+--------------+-------------+--------------+--------------+---------+------------------------+------------------------+-------------------+--------+-----------+-----------+------------------+------------------+-----------+-----------+---------------+---------------+---------------------+---------------------+----------------------+----------------------+
|temporada|       divisao|      data|     inicio_partida|     time_casa|    time_fora|gols_time_casa|gols_time_fora|resultado|time_casa_gols_intervalo|time_fora_gols_intervalo|resultado_intervalo| arbitro|chutes_casa|chutes_fora|chutes_no_gol_casa|chutes_no_gol_fora|faltas_casa|faltas_fora|escanteios_casa|escanteios_fora|cartoes_amarelos_casa|cartoes_amarelos_fora|cartoes_vermelhos_casa|cartoes_vermelhos_fora|
+---------+--------------+----------+-------------------+--------------+-------------+--------------+--------------+---------+------------------------+---------------------

## 4 Análise Descritiva e Exploratória

### 4.1 Calculando dados de precisões dos chutes

In [9]:
# precisão dos chutes no gol do time de casa e de fora
df = df.withColumn('precisao_chutes_casa', df['chutes_no_gol_casa'] / df['chutes_casa'])
df = df.withColumn('precisao_chutes_fora', df['chutes_no_gol_fora'] / df['chutes_fora'])
df.select('time_casa', 'time_fora', 'chutes_no_gol_casa', 'chutes_casa', 'precisao_chutes_casa', 'chutes_no_gol_fora', 'chutes_fora', 'precisao_chutes_fora').show(5)

+--------------+-------------+------------------+-----------+--------------------+------------------+-----------+--------------------+
|     time_casa|    time_fora|chutes_no_gol_casa|chutes_casa|precisao_chutes_casa|chutes_no_gol_fora|chutes_fora|precisao_chutes_fora|
+--------------+-------------+------------------+-----------+--------------------+------------------+-----------+--------------------+
|Crystal Palace|      Arsenal|               2.0|       10.0|                 0.2|               2.0|       10.0|                 0.2|
|        Fulham|    Liverpool|               3.0|        9.0|  0.3333333333333333|               4.0|       11.0| 0.36363636363636365|
|   Bournemouth|  Aston Villa|               3.0|        7.0| 0.42857142857142855|               2.0|       15.0| 0.13333333333333333|
|         Leeds|       Wolves|               4.0|       12.0|  0.3333333333333333|               6.0|       15.0|                 0.4|
|     Newcastle|Nott'm Forest|              10.0|      

### 4.2 Calculando média de gols por partida, temporada e divisão

In [10]:
# média de gols por partida, temporada e divisão ordenado por media de gols decrecente
from pyspark.sql.functions import avg

df_media_gols = df.groupBy('temporada', 'divisao') \
    .agg(avg('gols_time_casa').alias('media_gols_casa'), avg('gols_time_fora').alias('media_gols_fora')) \
    .orderBy('media_gols_casa', ascending=False)

df_media_gols.show(5)

+---------+------------+------------------+------------------+
|temporada|     divisao|   media_gols_casa|   media_gols_fora|
+---------+------------+------------------+------------------+
|    22-23|  Bundesliga|1.8562091503267975|1.3169934640522876|
|    01-02|Bundesliga 2|1.8431372549019607|1.3006535947712419|
|    03-04|  Bundesliga|1.8071895424836601|1.1633986928104576|
|    18-19|  Bundesliga|1.7908496732026145|1.3888888888888888|
|    96-97|  Bundesliga|1.7810457516339868| 1.196078431372549|
+---------+------------+------------------+------------------+
only showing top 5 rows



### 4.3 Calculando média de chutes no gol por partida, temporada e divisão

In [11]:
# média de chutes por partida, temporada e divisão ordenado por media de chutes decrecente
df_media_chutes = df.groupBy('temporada', 'divisao') \
    .agg(avg('chutes_casa').alias('media_chutes_casa'), avg('chutes_fora').alias('media_chutes_fora')) \
    .orderBy('media_chutes_casa', ascending=False)

df_media_chutes.show(5)

+---------+----------+------------------+------------------+
|temporada|   divisao| media_chutes_casa| media_chutes_fora|
+---------+----------+------------------+------------------+
|    03-04|Bundesliga|16.035947712418302|12.277777777777779|
|    05-06|Bundesliga|15.892156862745098|12.624183006535947|
|    07-08|Bundesliga|15.764705882352942|12.441176470588236|
|    04-05|Bundesliga| 15.50326797385621|12.058823529411764|
|    06-07|Bundesliga| 15.38562091503268|12.281045751633988|
+---------+----------+------------------+------------------+
only showing top 5 rows



### 4.4 Quantidade de vitórias, derrotas e empates por temporada e time

In [12]:
# quantidade de vitórias, empates e derrotas por time por temporada

df_vitorias = df.filter(df['resultado'] == 'Casa').groupBy('temporada', 'time_casa').count().withColumnRenamed('count', 'vitorias')
df_empates = df.filter(df['resultado'] == 'Empate').groupBy('temporada', 'time_casa').count().withColumnRenamed('count', 'empates')
df_derrotas = df.filter(df['resultado'] == 'Fora').groupBy('temporada', 'time_casa').count().withColumnRenamed('count', 'derrotas')

In [13]:

df_empates.orderBy('empates', ascending=False).show(3)
df_vitorias.orderBy('vitorias', ascending=False).show(3)
df_derrotas.orderBy('derrotas', ascending=False).show(3)


+---------+---------+-------+
|temporada|time_casa|empates|
+---------+---------+-------+
|    03-04|   Napoli|     15|
|    93-94|  Grimsby|     14|
|    09-10|  Vicenza|     13|
+---------+---------+-------+
only showing top 3 rows

+---------+---------+--------+
|temporada|time_casa|vitorias|
+---------+---------+--------+
|    13-14| Juventus|      19|
|    01-02| Man City|      19|
|    20-21|  Watford|      19|
+---------+---------+--------+
only showing top 3 rows

+---------+---------+--------+
|temporada|time_casa|derrotas|
+---------+---------+--------+
|    01-02|Stockport|      17|
|    03-04|Wimbledon|      16|
|    18-19|   Bolton|      15|
+---------+---------+--------+
only showing top 3 rows



## 5 Armazenando os dados 

### 5.1 Médias de chutes e gols por temporada e divisão

In [15]:
# dalve df_media_chutes, df_media_gols em arquivo parquet particionado por temporada e divisão e df_vitorias, df_empates, df_derrotas em arquivo parquet particionado por temporada na pasta datasink

df_media_chutes.write.partitionBy('temporada', 'divisao').parquet('datasink/media_chutes', mode='overwrite')
df_media_gols.write.partitionBy('temporada', 'divisao').parquet('datasink/media_gols', mode='overwrite')


### 5.2 Vitórias, empates e derrotas por temporada

In [14]:
# salve o df_vitorias, df_empates e df_derrotas em um arquivo parquet na pasta datasink particionada por temporada
df_vitorias.write.mode('overwrite').partitionBy('temporada').parquet('datasink/vitorias')
df_empates.write.mode('overwrite').partitionBy('temporada').parquet('datasink/empates')
df_derrotas.write.mode('overwrite').partitionBy('temporada').parquet('datasink/derrotas')


24/04/05 20:06:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
